In [ ]:
from keras.datasets import cifar10
from keras.models import *
from keras.layers import *
import numpy as np
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import *
from keras.preprocessing import image
from keras import regularizers,optimizers
from keras.callbacks import LearningRateScheduler
 
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 50:
        lrate = 0.0005
    if epoch > 75:
        lrate = 0.0003
    if epoch > 100:
        lrate = 0.0001
    return lrate
 

In [ ]:
(train_data,test_data) = cifar10.load_data()

In [ ]:
x_train = train_data[0]
x_train =np.asarray([one/255 for one in x_train])
y_train = to_categorical(train_data[1])


x_test = test_data[0]
x_test = np.asarray([one/255 for one in x_test])
y_test_pure = test_data[1]
y_test =to_categorical(test_data[1])



In [ ]:
def conv2d_bn(x,
              filters,
              kernel_size,
              strides=1,
              padding='same',
              activation='relu',
              use_bias=True,
              name=None):
    
    x = layers.Conv2D(filters,
                      kernel_size,
                      strides=strides,
                      padding=padding,
                      use_bias=use_bias,
                      name=name)(x)
    if activation is not None:
        ac_name = None if name is None else name + '_ac'
        x = layers.Activation(activation, name=ac_name)(x)
    return x

In [ ]:
"""

my inception
img_input = Input(shape=(32,32,3))
layer1 =  Conv2D(16,(3,3),padding='same',use_bias=True)(img_input)
layer1 = Conv2D(32, (3, 3),padding='same',use_bias=True)(layer1)
layer1 = MaxPooling2D((2, 2))(layer1)


branch_1 = Conv2D(8,(1,1),padding='same',use_bias=True)(layer1)

branch_2 = Conv2D(16,(2,2),padding='same',use_bias=True)(layer1)
branch_2 = Conv2D(8,(2,2),padding='same',use_bias=True)(layer1)
branch_2 = MaxPool2D(1,1)(branch_2)

branch_3 = Conv2D(16,(1,1),padding='same',use_bias=True)(layer1)
branch_3 = Conv2D(32,(3,3),padding='same',use_bias=True)(branch_3)
branch_3 = Conv2D(8,(2,2),padding='same',use_bias=True)(branch_3)

branch_pool =AveragePooling2D((3, 3),strides=(1, 1),padding='same')(layer1)
branch_pool = Conv2D(8,(1,1),padding='same',use_bias=True)(branch_pool)

concat_conv = concatenate([branch_1,branch_2,branch_3,branch_pool],axis=1)

branch_1 = Conv2D(8,(1,1),padding='same',use_bias=True)(layer1)

branch_2 = Conv2D(32,(2,2),padding='same',use_bias=True)(layer1)
branch_2 = Conv2D(16,(2,2),padding='same',use_bias=True)(layer1)
branch_2 = MaxPool2D(1,1)(branch_2)

branch_3 = Conv2D(64,(1,1),padding='same',use_bias=True)(layer1)
branch_3 = Conv2D(96,(3,3),padding='same',use_bias=True)(branch_3)
branch_3 = Conv2D(16,(2,2),padding='same',use_bias=True)(branch_3)

branch_pool =AveragePooling2D((3, 3),strides=(1, 1),padding='same')(layer1)
branch_pool = Conv2D(16,(1,1),padding='same',use_bias=True)(branch_pool)

concat_conv = concatenate([branch_1,branch_2,branch_3,branch_pool],axis=3)
flt = GlobalAveragePooling2D()(concat_conv)
Den = Dense(256,activation='relu')(flt)
Den = Dropout(0.5)(Den)
res = Dense(100,activation='softmax')(Den)

model = Model(img_input,res)
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['acc'])
model.summary()
"""

In [ ]:
def make_conv_model_list():
    weight_decay = 1e-4
    model = Sequential()
    model.add(Conv2D(32,(2,2),use_bias=True,activation='relu',padding='same',input_shape=(32,32,3),kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(2,2))
    model.add(BatchNormalization())
    model.add(ZeroPadding2D(padding=(1, 1), data_format=None))
    model.add(Conv2D(32,(4,4),use_bias=True,padding='same',activation='relu',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(2,2))
    model.add(BatchNormalization())
    model.add(ZeroPadding2D(padding=(1, 1), data_format=None))
    model.add(Conv2D(64,(2,2),use_bias=True,padding='same',activation='relu',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(2,2))
    model.add(Flatten())
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    opt_rms = optimizers.rmsprop(lr=0.001,decay=1e-6)
    model.compile(optimizer=opt_rms,loss='categorical_crossentropy',metrics=['acc'])
    model.summary()
    return model

In [ ]:
ensemble_model_list = []
ensemble_size=5
ensemble_call_back =[EarlyStopping(monitor='val_acc', min_delta=0, patience=25, verbose=1, mode='auto'),LearningRateScheduler(lr_schedule)]
for i in range(ensemble_size):
    this_model = make_conv_model_list()
    print('i_th model training')
    this_model.fit(x_train, y_train,
          batch_size=128,
          epochs=150,callbacks=ensemble_call_back,
          verbose=1,
          validation_data=(x_test, y_test))
    
    ensemble_model_list.append(this_model)


In [ ]:
all_predicted = np.zeros(y_test.shape)

In [ ]:
for one in ensemble_model_list:
    all_predicted =all_predicted+one.predict(x_test)

In [ ]:
final_res = []
for one in all_predicted:
    final_res.append(np.argmax(one))

In [ ]:
final_res = np.asarray(final_res)

In [ ]:
entire = y_test_pure.shape[0]

In [ ]:
ele = 0
for one in range(entire):
    if(y_test_pure[one]==final_res[one]):
        ele=ele+1
    
res_acc = ele/entire

In [ ]:
res_acc